In [4]:
# === 一键跑：修复模型 → 打包数据(若缺) → 训练(早停) → 最终测试报告 & 混淆矩阵 ===
import os, numpy as np, torch, torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# 从你的 pipeline 导入这些已有工具（它们在你的 stgcn_fiveclass_pipeline.py 里）
from stgcn_fiveclass_pipeline import (
    Graph, STGCNWindowDataset, DefaultPaths, pack_train_val_test, FIVE_LABELS
)

device = "cuda" if torch.cuda.is_available() else "cpu"

# ----------------------------
# 1) 修复版 GCN / STGCN（通道数对齐：3→64→...）
# ----------------------------
class GCN_Fix(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, A: np.ndarray, num_subset: int = 3):
        super().__init__()
        self.register_buffer('A', torch.tensor(A, dtype=torch.float32))  # (3,V,V)
        self.num_subset = num_subset
        # 用 1x1 卷积直接把 in_channels → out_channels，避免通道不匹配
        self.convs = nn.ModuleList([nn.Conv2d(in_channels, out_channels, 1) for _ in range(num_subset)])
        self.bn = nn.BatchNorm2d(out_channels)
        self.down = nn.Identity() if (in_channels == out_channels) else nn.Conv2d(in_channels, out_channels, 1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):  # x: (N,C,T,V)
        y = None
        A = self.A
        for i in range(self.num_subset):
            z = torch.einsum('nctv,vw->nctw', x, A[i])  # 图聚合，不改变通道数
            z = self.convs[i](z)                       # 通道对齐到 out_channels
            y = z if y is None else y + z
        y = self.bn(y)
        y = y + self.down(x)
        return self.relu(y)

class STGCN_Compat(nn.Module):
    def __init__(self, in_channels: int, num_class: int, A: np.ndarray):
        super().__init__()
        V = A.shape[-1]
        self.data_bn = nn.BatchNorm1d(in_channels * V)
        # GCN + TCN
        self.g1 = GCN_Fix(in_channels, 64, A);  self.t1 = nn.Sequential(nn.Conv2d(64,64,(9,1),padding=(4,0)), nn.BatchNorm2d(64), nn.ReLU(True))
        self.g2 = GCN_Fix(64, 64, A);           self.t2 = nn.Sequential(nn.Conv2d(64,64,(9,1),padding=(4,0)), nn.BatchNorm2d(64), nn.ReLU(True))
        self.g3 = GCN_Fix(64,128, A);           self.t3 = nn.Sequential(nn.Conv2d(128,128,(9,1),padding=(4,0)), nn.BatchNorm2d(128), nn.ReLU(True))
        self.g4 = GCN_Fix(128,256, A);          self.t4 = nn.Sequential(nn.Conv2d(256,256,(9,1),padding=(4,0)), nn.BatchNorm2d(256), nn.ReLU(True))
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)), nn.Flatten(), nn.Dropout(0.5), nn.Linear(256, num_class))

    def forward(self, x):  # x: (N,3,T,32)
        N, C, T, V = x.shape
        x = x.permute(0,2,1,3).contiguous().view(N, T, C*V)
        x = self.data_bn(x.transpose(1,2)).transpose(1,2)
        x = x.view(N, T, C, V).permute(0,2,1,3).contiguous()
        x = self.t1(self.g1(x))
        x = self.t2(self.g2(x))
        x = self.t3(self.g3(x))
        x = self.t4(self.g4(x))
        return self.head(x)

# ----------------------------
# 2) 若打包数据不存在，则自动打包（用你刚导出的三份标签）
# ----------------------------
paths = DefaultPaths(
    lab_16gz="data/labels_16GZ.npy",
    lab_17jp="data/labels_17JP.npy",
    lab_19mm="data/labels_19MM.npy",
)
need_pack = any(not os.path.isfile(p) for p in [paths.out_train, paths.out_val, paths.out_test])
if need_pack:
    print("⚙️ 未找到打包文件，正在打包 train/val/test ...")
    pack_train_val_test(win=120, stride=30, strict=True, mode="center", paths=paths)

# ----------------------------
# 3) DataLoader
# ----------------------------
train_ds = STGCNWindowDataset(paths.out_train)
val_ds   = STGCNWindowDataset(paths.out_val)
test_ds  = STGCNWindowDataset(paths.out_test)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=0)

# ----------------------------
# 4) 训练（早停指标：Val macro-F1）
# ----------------------------
A = Graph().A
model = STGCN_Compat(in_channels=3, num_class=len(FIVE_LABELS), A=A).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
crit = nn.CrossEntropyLoss()
patience = 5
best_val, best_state, wait = -1.0, None, 0

def eval_loader(loader):
    model.eval(); yT=[]; yP=[]; L=0.0; N=0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            logit = model(x); loss = crit(logit,y)
            L += float(loss.item())*y.numel(); N += int(y.numel())
            yT.append(y.cpu().numpy()); yP.append(logit.argmax(1).cpu().numpy())
    yT = np.concatenate(yT) if yT else np.array([],dtype=np.int64)
    yP = np.concatenate(yP) if yP else np.array([],dtype=np.int64)
    f1 = f1_score(yT, yP, average="macro") if len(yT)>0 else 0.0
    return (L/max(1,N)), f1, yT, yP

for ep in range(1, 41):
    model.train(); runL=0.0; N=0; ytr=[]; ypr=[]
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        logit = model(x); loss = crit(logit,y)
        opt.zero_grad(); loss.backward(); opt.step()
        runL += float(loss.item())*y.numel(); N += int(y.numel())
        ytr.append(y.cpu().numpy()); ypr.append(logit.argmax(1).detach().cpu().numpy())
    trF = f1_score(np.concatenate(ytr), np.concatenate(ypr), average="macro") if N>0 else 0.0
    _, vaF, yv, pv = eval_loader(val_loader)
    _, teF, yt, pt = eval_loader(test_loader)
    print(f"Epoch {ep:02d}/40 | Train F1: {trF:.4f} | Val F1: {vaF:.4f} | Test F1: {teF:.4f} | L: {runL/max(1,N):.4f}")
    if vaF > best_val + 1e-5:
        best_val, best_state, wait = vaF, {k:v.cpu() for k,v in model.state_dict().items()}, 0
    else:
        wait += 1
        if wait >= patience:
            print("⚠️ Early stop"); break

if best_state is not None:
    model.load_state_dict(best_state)

# ----------------------------
# 5) 最终测试报告（稳定版：显式指定 labels=range(5)）
# ----------------------------
_, _, ytrue, ypred = eval_loader(test_loader)  # 重新评估获得最终预测
names = [f"a_{n}" for n in FIVE_LABELS]
all_labels = list(range(len(FIVE_LABELS)))
print("\n🎯 最终测试集性能 (Test Set):\n")
print(classification_report(
    ytrue, ypred,
    labels=all_labels,
    target_names=names,
    digits=2,
    zero_division=0
))
print("混淆矩阵:")
print(confusion_matrix(ytrue, ypred, labels=all_labels))


Epoch 01/40 | Train F1: 0.2638 | Val F1: 0.1250 | Test F1: 0.1708 | L: 1.0165
Epoch 02/40 | Train F1: 0.2826 | Val F1: 0.1398 | Test F1: 0.1874 | L: 0.8945
Epoch 03/40 | Train F1: 0.2996 | Val F1: 0.1506 | Test F1: 0.1890 | L: 0.8497
Epoch 04/40 | Train F1: 0.3378 | Val F1: 0.1121 | Test F1: 0.1447 | L: 0.8171
Epoch 05/40 | Train F1: 0.3930 | Val F1: 0.1820 | Test F1: 0.2463 | L: 0.7772
Epoch 06/40 | Train F1: 0.4129 | Val F1: 0.1939 | Test F1: 0.2100 | L: 0.7600
Epoch 07/40 | Train F1: 0.4526 | Val F1: 0.1308 | Test F1: 0.2385 | L: 0.7405
Epoch 08/40 | Train F1: 0.4653 | Val F1: 0.1329 | Test F1: 0.2122 | L: 0.7298
Epoch 09/40 | Train F1: 0.5187 | Val F1: 0.1437 | Test F1: 0.2644 | L: 0.6892
Epoch 10/40 | Train F1: 0.5362 | Val F1: 0.1739 | Test F1: 0.2474 | L: 0.6798
Epoch 11/40 | Train F1: 0.5188 | Val F1: 0.1496 | Test F1: 0.2043 | L: 0.6798
⚠️ Early stop

🎯 最终测试集性能 (Test Set):

              precision    recall  f1-score   support

      a_walk       0.48      0.07      0.13      

In [3]:
# === Fix window_label 安全处理空窗口 + 重新打包（win=15, stride=5, strict=False, center） ===
import stgcn_fiveclass_pipeline as p
from collections import Counter

# 1) 猴子补丁：替换原 window_label（避免中心/多数投票在空列表时崩溃）
def window_label_fix(window_labels, mode: str = "center"):
    # window_labels: 长度=win 的字符串/None 列表
    if mode == "center":
        # 允许空或中心为 None 时返回 None
        if not window_labels:
            return None
        mid = len(window_labels) // 2
        v = window_labels[mid]
        return v if (v in p.FIVE_SET) else None
    else:  # "majority"
        valid = [v for v in window_labels if v in p.FIVE_SET]
        if not valid:
            return None
        return Counter(valid).most_common(1)[0][0]

# 应用补丁
p.window_label = window_label_fix

# 2) 重新打包（短窗口）
paths = p.DefaultPaths(
    lab_16gz="data/labels_16GZ.npy",
    lab_17jp="data/labels_17JP.npy",
    lab_19mm="data/labels_19MM.npy",
)
p.pack_train_val_test(
    win=15,
    stride=5,
    strict=False,     # 只要求中心帧为五类
    mode="center",    # 使用中心帧作为窗口标签（已安全处理）
    paths=paths
)

print("✅ Repacked with win=15, stride=5, strict=False, mode='center'.")


[pack] train: (2729, 3, 15, 32, 1) (2729,)
[pack] val  : (1874, 3, 15, 32, 1) (1874,)
[pack] test : (2448, 3, 15, 32, 1) (2448,)
✅ Repacked with win=15, stride=5, strict=False, mode='center'.


In [6]:
from stgcn_fiveclass_pipeline import make_windows_stgcn_from_paths, FIVE_LABELS
import numpy as np

x_path = "data/poses_09SY_cs_nz.npy"
lab_out = "data/labels_09SY.npy"          # ← 用原始的，不要 *_kept.npy
k_idx  = "data/poses_09SY_kept_idx.npy"
win, stride, strict, mode = 15, 5, False, "center"

Xseq, Yseq = make_windows_stgcn_from_paths(
    x_path, lab_out, k_idx, win=win, stride=stride, strict=strict, mode=mode
)
np.savez_compressed("data/test_09sy_stgcn.npz", X=Xseq, Y=Yseq, meta=dict(labels=FIVE_LABELS))
print("[OK] 09SY windows:", Xseq.shape, Yseq.shape)


[OK] 09SY windows: (1916, 3, 15, 32, 1) (1916,)


In [7]:
# === 合并 16GZ/17JP/19MM/09SY → 统一窗口化 → 分层随机划分 Train/Val/Test (70/15/15) ===
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from stgcn_fiveclass_pipeline import make_windows_stgcn_from_paths, FIVE_LABELS

# 统一窗口参数（与你当前流程一致）
WIN, STRIDE, STRICT, MODE = 15, 5, False, "center"

# 四个数据源的路径（已完成：中心化/尺度归一/去零帧 + 原始逐帧标签 + kept_idx）
SOURCES = [
    dict(name="16GZ",
         x="data/poses_16GZ_cs_nz.npy",
         y="data/labels_16GZ.npy",
         k="data/poses_16GZ_kept_idx.npy"),
    dict(name="17JP",
         x="data/poses_17JP_cs_nz.npy",
         y="data/labels_17JP.npy",
         k="data/poses_17JP_kept_idx.npy"),
    dict(name="19MM",
         x="data/poses_19MM_cs_nz.npy",
         y="data/labels_19MM.npy",
         k="data/poses_19MM_kept_idx.npy"),
    dict(name="09SY",
         x="data/poses_09SY_cs_nz.npy",
         y="data/labels_09SY.npy",
         k="data/poses_09SY_kept_idx.npy"),
]

def pack_one(src):
    X, Y = make_windows_stgcn_from_paths(
        src["x"], src["y"], src["k"],
        win=WIN, stride=STRIDE, strict=STRICT, mode=MODE
    )
    # Y 是( N, ) 的对象数组（五类字符串名或已映射为id，取决于你的实现）
    # 统一转成类别 id（0..4）
    name2id = {n:i for i,n in enumerate(FIVE_LABELS)}
    if Y.dtype.kind in ("U","O","S"):  # 字符串
        Yid = np.array([name2id[y] for y in Y], dtype=np.int64)
    else:
        Yid = Y.astype(np.int64)
    print(f"[{src['name']}] X={X.shape}, Y={Yid.shape}, class_cnt={Counter(Yid)}")
    return X, Yid

# 1) 分别窗口化并汇总
Xs, Ys = [], []
for s in SOURCES:
    Xi, Yi = pack_one(s)
    Xs.append(Xi); Ys.append(Yi)

X_all = np.concatenate(Xs, axis=0)       # (N, 3, T, 32, 1)
Y_all = np.concatenate(Ys, axis=0)       # (N,)
print(f"\n[ALL] X={X_all.shape}, Y={Y_all.shape}, class_cnt={Counter(Y_all)}")

# 2) 分层划分 Train/Val/Test = 70/15/15
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X_all, Y_all, test_size=0.15, random_state=42, stratify=Y_all
)
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp, test_size=0.1764706, random_state=42, stratify=y_tmp
)
# 备注：0.1764706 ≈ 0.15 / 0.85，使得最终 70/15/15

def cnt(y): return dict(sorted(Counter(y).items()))
print("\n[SPLIT]")
print(" Train:", X_train.shape, cnt(y_train))
print(" Val  :", X_val.shape,   cnt(y_val))
print(" Test :", X_test.shape,  cnt(y_test))

# 3) 保存到标准文件名，供训练代码直接使用
np.savez_compressed("data/train_stgcn.npz", X=X_train, Y=y_train, meta=dict(labels=FIVE_LABELS))
np.savez_compressed("data/val_stgcn.npz",   X=X_val,   Y=y_val,   meta=dict(labels=FIVE_LABELS))
np.savez_compressed("data/test_stgcn.npz",  X=X_test,  Y=y_test,  meta=dict(labels=FIVE_LABELS))
print("\n[OK] Saved to data/train_stgcn.npz, data/val_stgcn.npz, data/test_stgcn.npz")


[16GZ] X=(2729, 3, 15, 32, 1), Y=(2729,), class_cnt=Counter({np.int64(2): 1585, np.int64(3): 591, np.int64(0): 420, np.int64(4): 96, np.int64(1): 37})
[17JP] X=(1874, 3, 15, 32, 1), Y=(1874,), class_cnt=Counter({np.int64(3): 625, np.int64(2): 619, np.int64(0): 530, np.int64(4): 77, np.int64(1): 23})
[19MM] X=(2448, 3, 15, 32, 1), Y=(2448,), class_cnt=Counter({np.int64(2): 1037, np.int64(3): 540, np.int64(0): 530, np.int64(4): 188, np.int64(1): 153})
[09SY] X=(1916, 3, 15, 32, 1), Y=(1916,), class_cnt=Counter({np.int64(2): 894, np.int64(3): 840, np.int64(0): 117, np.int64(4): 65})

[ALL] X=(8967, 3, 15, 32, 1), Y=(8967,), class_cnt=Counter({np.int64(2): 4135, np.int64(3): 2596, np.int64(0): 1597, np.int64(4): 426, np.int64(1): 213})

[SPLIT]
 Train: (6276, 3, 15, 32, 1) {np.int64(0): 1118, np.int64(1): 149, np.int64(2): 2894, np.int64(3): 1817, np.int64(4): 298}
 Val  : (1345, 3, 15, 32, 1) {np.int64(0): 239, np.int64(1): 32, np.int64(2): 620, np.int64(3): 390, np.int64(4): 64}
 Test : 

In [8]:
# === Oversample lie(1) & bend(4) in TRAIN set, save to train_stgcn_over.npz ===
import numpy as np
from collections import Counter

npz_in  = "data/train_stgcn.npz"
npz_out = "data/train_stgcn_over.npz"

# 标签编号（如与你不一致请改这里）
SIT, LIE, STAND, WALK, BEND = 0, 1, 2, 3, 4

dat = np.load(npz_in, allow_pickle=True)
X, Y = dat["X"], dat["Y"]
labels = dat["meta"].item().get("labels", ["sit","lie","stand","walk","bend"])

def show_cnt(tag, y):
    c = Counter(y.tolist())
    ordered = {k:c.get(k,0) for k in range(5)}
    print(f"{tag}: {ordered}")
    return ordered

print("== Before ==")
cnt_before = show_cnt("train", Y)

# 目标规模：把少数类提升到多数类的 ~80%-90%
max_c = max(cnt_before.values()) if cnt_before else 0
target = cnt_before.copy()
target[LIE]  = min(int(max_c * 0.85), max_c)   # lie 目标≈多数类的85%
target[BEND] = min(int(max_c * 0.90), max_c)   # bend 目标≈多数类的90%

# 计算每类需要的重复倍数（上限避免过度复制）
R_MAX = 10
repeats = {k:1 for k in range(5)}
for k in [LIE, BEND]:
    n = cnt_before.get(k,0)
    if n > 0:
        repeats[k] = min(R_MAX, int(np.ceil(target[k] / n)))
    else:
        repeats[k] = 1  # 没有该类就不重复（可考虑数据增强补齐）

# 构造过采样后的索引
idx_all = np.arange(len(Y))
idx_per_class = {k: idx_all[Y == k] for k in range(5)}

new_idx = []
rng = np.random.default_rng(42)

for k in range(5):
    idx_k = idx_per_class[k]
    if len(idx_k) == 0:
        continue
    rep = repeats[k]
    if rep <= 1:
        new_idx.append(idx_k)
    else:
        # 重复 rep 次后，若超出目标数量则随机截断到 target[k]
        tiled = np.tile(idx_k, rep)
        if k in (LIE, BEND):
            need = target[k]
            if len(tiled) > need:
                tiled = rng.choice(tiled, size=need, replace=False)
        new_idx.append(tiled)

# 其他类别（未指定目标）的样本直接加入
#（上面已加入；若想严格对齐到同一规模，可同样设置 target 并截断）

new_idx = np.concatenate(new_idx)
rng.shuffle(new_idx)

X_over = X[new_idx]
Y_over = Y[new_idx]

print("\n== After  (oversampled) ==")
cnt_after = show_cnt("train_over", Y_over)

# 保存
np.savez_compressed(npz_out, X=X_over, Y=Y_over, meta=dict(labels=labels))
print(f"\n[OK] Saved oversampled train -> {npz_out}")


== Before ==
train: {0: 1118, 1: 149, 2: 2894, 3: 1817, 4: 298}

== After  (oversampled) ==
train_over: {0: 1118, 1: 1490, 2: 2894, 3: 1817, 4: 2604}

[OK] Saved oversampled train -> data/train_stgcn_over.npz


In [2]:
# ==== ST-GCN：自动对齐每块输入通道（按上一块输出）+ K=1 图 + 3→16 适配 + 训练/评估 ====
import os, numpy as np, torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, confusion_matrix

# 你的 pipeline
import stgcn_fiveclass_pipeline as p
STGCN              = p.STGCN
STGCNWindowDataset = p.STGCNWindowDataset
Graph              = p.Graph
FIVE_LABELS        = p.FIVE_LABELS

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ---------- 数据 ----------
train_path = "data/train_stgcn_over.npz" if os.path.exists("data/train_stgcn_over.npz") else "data/train_stgcn.npz"
val_path   = "data/val_stgcn.npz"
test_path  = "data/test_stgcn.npz"

train_ds = STGCNWindowDataset(train_path)
val_ds   = STGCNWindowDataset(val_path)
test_ds  = STGCNWindowDataset(test_path)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  drop_last=False)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, drop_last=False)

# ---------- 图：合并多子图为单图(K=1) ----------
A = np.asarray(Graph().A, dtype=np.float32)
if A.ndim == 2:
    A = A[None, ...]
elif A.ndim == 3 and A.shape[0] != 1:
    A = A.sum(axis=0, keepdims=True)
print("Adjacency (merged):", A.shape)

# ---------- 输入适配：3 → 16 ----------
class InputAdapter(nn.Module):
    def __init__(self, in_ch=3, out_ch=16):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_ch)
        )
    def forward(self, x):  # (N,3,T,V)
        return self.proj(x)

adapter = InputAdapter(3, 16).to(device)

# ---------- 模型本体 ----------
model_core = STGCN(in_channels=16, num_class=len(FIVE_LABELS), A=A).to(device)

# ---------- 补丁①：A 复制到每个 GCN 的 num_subset ----------
def _replicate_A_for_module_A(A_np, num_subset):
    if A_np.ndim == 2:
        A_np = A_np[None, ...]
    if A_np.shape[0] != num_subset:
        A_np = np.repeat(A_np, num_subset, axis=0)
    return torch.tensor(A_np, dtype=torch.float32)

for m in model_core.modules():
    if m.__class__.__name__ == "GCN" and hasattr(m, "A") and hasattr(m, "num_subset"):
        m.A = _replicate_A_for_module_A(A, m.num_subset).to(device)

# ---------- 补丁②：自动把 g2/g3/g4 的 conv_d 输入通道，改成“上一块输出通道” ----------
def _set_conv_in_channels(conv2d: nn.Conv2d, in_ch: int) -> nn.Conv2d:
    # 保留 kernel/stride/padding/dilation/out_channels 等，替换 in_channels
    new_conv = nn.Conv2d(
        in_channels=in_ch,
        out_channels=conv2d.out_channels,
        kernel_size=conv2d.kernel_size,
        stride=conv2d.stride,
        padding=conv2d.padding,
        dilation=conv2d.dilation,
        groups=conv2d.groups,
        bias=(conv2d.bias is not None),
    )
    return new_conv

def _block_out_channels(gcn_module) -> int:
    # 以第一支 conv_d 的 out_channels 作为该块输出通道（你的实现里各支一致）
    return int(gcn_module.conv_d[0].out_channels)

# 依次列出块（如有就取）
blocks = [getattr(model_core, n) for n in ("g1","g2","g3","g4") if hasattr(model_core, n)]
# 第一块的输入通道等于适配器输出 16，不改；从第二块开始对齐输入通道 = 上一块输出通道
prev_out = _block_out_channels(blocks[0])  # g1 的 out_channels，通常 64
for blk in blocks[1:]:
    # 把本块的每个 conv_d 的 in_channels 改成 prev_out
    new_list = []
    for conv in blk.conv_d:
        new_list.append(_set_conv_in_channels(conv, prev_out).to(device))
    # 替换 ModuleList
    blk.conv_d = nn.ModuleList(new_list)
    # 更新 prev_out = 当前块的输出通道
    prev_out = _block_out_channels(blk)

# ---------- 包装最终模型：先适配通道再过核心 ----------
class WrappedSTGCN(nn.Module):
    def __init__(self, adapter, core):
        super().__init__()
        self.adapter = adapter
        self.core = core
    def forward(self, x):
        x = self.adapter(x)
        return self.core(x)

model = WrappedSTGCN(adapter, model_core).to(device)

# ---------- 类别权重 ----------
y_train = np.load(train_path)["Y"]
from collections import Counter
cnt = Counter(y_train.tolist()); tot = sum(cnt.values())
weights = np.array([tot / max(cnt.get(i,1),1) for i in range(len(FIVE_LABELS))], dtype=np.float32)
weights = weights / weights.mean()
print("Class weights:", weights.round(2).tolist())
crit = nn.CrossEntropyLoss(weight=torch.tensor(weights, dtype=torch.float32, device=device))
opt  = torch.optim.Adam(model.parameters(), lr=5e-4)
EPOCHS, PATIENCE = 80, 12

# ---------- 评估 ----------
@torch.no_grad()
def eval_loader(loader):
    model.eval()
    yT, yP = [], []
    for x,y in loader:
        x,y = x.to(device), y.to(device)
        out = model(x)
        yP.append(out.argmax(1).cpu().numpy())
        yT.append(y.cpu().numpy())
    yT = np.concatenate(yT); yP = np.concatenate(yP)
    return f1_score(yT, yP, average="macro", zero_division=0), yT, yP

# ---------- 训练（早停+保存最优） ----------
best_val, best_state, wait = -1.0, None, 0
for ep in range(1, EPOCHS+1):
    model.train()
    yT, yP = [], []
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        out = model(x)
        loss = crit(out, y)
        opt.zero_grad(); loss.backward(); opt.step()
        yT.append(y.detach().cpu().numpy())
        yP.append(out.argmax(1).detach().cpu().numpy())
    yT = np.concatenate(yT); yP = np.concatenate(yP)
    tr_f1 = f1_score(yT, yP, average="macro", zero_division=0)

    va_f1,_,_ = eval_loader(val_loader)
    te_f1,_,_ = eval_loader(test_loader)
    print(f"Epoch {ep:02d}/{EPOCHS} | Train F1: {tr_f1:.4f} | Val F1: {va_f1:.4f} | Test F1: {te_f1:.4f}")

    if va_f1 > best_val:
        best_val = va_f1
        best_state = {k:v.cpu() for k,v in model.state_dict().items()}
        wait = 0
    else:
        wait += 1
        if wait >= PATIENCE:
            print("⚠️ Early stop"); break

# ---------- 保存/测试 ----------
if best_state is not None:
    model.load_state_dict(best_state)
os.makedirs("data", exist_ok=True)
torch.save(model.state_dict(), "data/best_stgcn.pth")
print(f"✅ Best Val F1: {best_val:.4f} | Weights saved to data/best_stgcn.pth")

te_f1, ytrue, ypred = eval_loader(test_loader)
print("\n🎯 Test macro-F1:", round(te_f1,4))
names = [f"a_{n}" for n in FIVE_LABELS]
print("\nClassification report:")
print(classification_report(ytrue, ypred, labels=list(range(len(FIVE_LABELS))), target_names=names, digits=2, zero_division=0))
print("Confusion matrix:")
print(confusion_matrix(ytrue, ypred, labels=list(range(len(FIVE_LABELS)))))


Device: cpu
Adjacency (merged): (1, 32, 32)
Class weights: [1.5700000524520874, 1.1799999475479126, 0.6100000143051147, 0.9700000286102295, 0.6700000166893005]
Epoch 01/80 | Train F1: 0.4151 | Val F1: 0.3006 | Test F1: 0.3054
Epoch 02/80 | Train F1: 0.4913 | Val F1: 0.4090 | Test F1: 0.4122
Epoch 03/80 | Train F1: 0.5276 | Val F1: 0.3849 | Test F1: 0.3914
Epoch 04/80 | Train F1: 0.5410 | Val F1: 0.4695 | Test F1: 0.4446
Epoch 05/80 | Train F1: 0.5672 | Val F1: 0.4815 | Test F1: 0.4622
Epoch 06/80 | Train F1: 0.5865 | Val F1: 0.4561 | Test F1: 0.4384
Epoch 07/80 | Train F1: 0.6123 | Val F1: 0.4737 | Test F1: 0.4881
Epoch 08/80 | Train F1: 0.6286 | Val F1: 0.5040 | Test F1: 0.4822
Epoch 09/80 | Train F1: 0.6379 | Val F1: 0.5331 | Test F1: 0.5253
Epoch 10/80 | Train F1: 0.6571 | Val F1: 0.4112 | Test F1: 0.4011
Epoch 11/80 | Train F1: 0.6673 | Val F1: 0.5029 | Test F1: 0.5151
Epoch 12/80 | Train F1: 0.6737 | Val F1: 0.5528 | Test F1: 0.5660
Epoch 13/80 | Train F1: 0.6869 | Val F1: 0.5677 

### 